# Fabric Date Table Generator (T-SQL Approach)

This notebook generates a dynamic date table using T-SQL, suitable for Microsoft Fabric Lakehouse, Synapse, or Power BI Direct Lake scenarios.

## Parameters

Define the parameters for the date table generation. Adjust as needed:
- **@StartDate**: First date in the table (YYYY-MM-DD)
- **@EndDate**: Last date in the table (YYYY-MM-DD)
- **@FiscalYearStartMonth**: Month (1-12) when the fiscal year starts
- **@HolidayDates**: Comma-separated list of holiday dates (YYYY-MM-DD) to flag in the table

In [1]:
IF NOT EXISTS (
    SELECT 1 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'DateParameters'
)
BEGIN
    -- Create the table (or use CTAS if needed)
    CREATE TABLE dbo.DateParameters (
        StartDate DATE,
        EndDate DATE,
        FiscalYearStartMonth INT,
        HolidayDates VARCHAR(100)
    );

    INSERT INTO dbo.DateParameters
    VALUES (
        '2015-01-01',
        '2030-12-31',
        7,
        '2025-01-01,2025-12-25'
    );
END;

## Generate Date Table

In [2]:
IF NOT EXISTS (
    SELECT 1 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'DateRange'
)
BEGIN
    CREATE TABLE dbo.DateRange (
        [Date] DATE,
        [Year] INT,
        [Month] INT,
        [Day] INT,
        [Quarter] INT,
        [DayOfWeek] INT,
        [DayOfWeekName] VARCHAR(20)
    );
END;

### Insert the Date Range

In [3]:
-- Only insert if the table is empty
WITH Numbers AS (
    SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) - 1 AS n
    FROM (VALUES (1),(1),(1),(1),(1),(1),(1),(1),(1),(1)) AS t1(n)
    CROSS JOIN (VALUES (1),(1),(1),(1),(1),(1),(1),(1),(1),(1)) AS t2(n)
    CROSS JOIN (VALUES (1),(1),(1),(1),(1),(1),(1),(1),(1),(1)) AS t3(n)
),
DateRange AS (
    SELECT DATEADD(DAY, n, CAST('2015-01-01' AS DATE)) AS [Date]
    FROM Numbers
    WHERE DATEADD(DAY, n, CAST('2015-01-01' AS DATE)) <= '2030-12-31'
)
INSERT INTO dbo.DateRange ([Date], [Year], [Month], [Day], [Quarter], [DayOfWeek], [DayOfWeekName])
SELECT 
    [Date],
    YEAR([Date]),
    MONTH([Date]),
    DAY([Date]),
    DATEPART(QUARTER, [Date]),
    DATEPART(WEEKDAY, [Date]),
    DATENAME(WEEKDAY, [Date])
FROM DateRange;


## Add Calendar and Fiscal Columns

Add columns for year, month, quarter, fiscal year, fiscal month, and fiscal quarter.

In [4]:
-- Step 1: Create the enriched date table
IF NOT EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = 'dbo' 
      AND TABLE_NAME = 'DateTableEnriched'
)
BEGIN
    CREATE TABLE dbo.DateTableEnriched (
        [Date] DATE,
        [Year] INT,
        [Month] INT,
        [MonthName] VARCHAR(20),
        [Day] INT,
        [Quarter] INT,
        [WeekOfYear] INT,
        [DayOfWeekName] VARCHAR(20),
        [DayOfWeek] INT,
        [FiscalYear] INT,
        [FiscalMonth] INT,
        [FiscalQuarter] INT
    );
END;

In [5]:
-- Step 2: Insert enriched calendar and fiscal data
IF EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = 'dbo' 
      AND TABLE_NAME = 'DateRange'
)
BEGIN
    INSERT INTO dbo.DateTableEnriched
    SELECT
        [Date],
        YEAR([Date]) AS [Year],
        MONTH([Date]) AS [Month],
        DATENAME(MONTH, [Date]) AS [MonthName],
        DAY([Date]) AS [Day],
        DATEPART(QUARTER, [Date]) AS [Quarter],
        DATEPART(WEEK, [Date]) AS [WeekOfYear],
        DATENAME(WEEKDAY, [Date]) AS [DayOfWeekName],
        DATEPART(WEEKDAY, [Date]) AS [DayOfWeek],
        -- Fiscal logic (assuming fiscal year starts in July)
        CASE 
            WHEN MONTH([Date]) >= 7 THEN YEAR([Date]) + 1 
            ELSE YEAR([Date]) 
        END AS [FiscalYear],
        ((MONTH([Date]) - 7 + 12) % 12 + 1) AS [FiscalMonth],
        (((MONTH([Date]) - 7 + 12) % 12) / 3 + 1) AS [FiscalQuarter]
    FROM dbo.DateRange;
END;


## Flag Holidays and Current Dates

Add columns to flag holidays and current day/month/year.

In [6]:
-- Step 3: Create the final table if it doesn't exist
IF NOT EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = 'dbo' 
      AND TABLE_NAME = 'DateTableFinal'
)
BEGIN
    CREATE TABLE dbo.DateTableFinal (
        [Date] DATE,
        [Year] INT,
        [Month] INT,
        [MonthName] VARCHAR(20),
        [Day] INT,
        [Quarter] INT,
        [WeekOfYear] INT,
        [DayOfWeekName] VARCHAR(20),
        [DayOfWeek] INT,
        [FiscalYear] INT,
        [FiscalMonth] INT,
        [FiscalQuarter] INT,
        [IsHoliday] INT,
        [IsCurrentDay] INT,
        [IsCurrentMonth] INT,
        [IsCurrentYear] INT
    );
END;

-- Step 4: Insert enriched data with holiday and current date flags
-- Replace the holiday list below with your actual dates
INSERT INTO dbo.DateTableFinal
SELECT 
    d.*,
    CASE WHEN d.[Date] IN (
        SELECT TRY_CAST(value AS DATE) 
        FROM STRING_SPLIT('2025-01-01,2025-07-04,2025-12-25', ',')  -- Example holidays
    ) THEN 1 ELSE 0 END AS [IsHoliday],
    CASE WHEN d.[Date] = CAST(GETDATE() AS DATE) THEN 1 ELSE 0 END AS [IsCurrentDay],
    CASE WHEN d.[Year] = YEAR(GETDATE()) AND d.[Month] = MONTH(GETDATE()) THEN 1 ELSE 0 END AS [IsCurrentMonth],
    CASE WHEN d.[Year] = YEAR(GETDATE()) THEN 1 ELSE 0 END AS [IsCurrentYear]
FROM dbo.DateTableEnriched d;

## Write Date Table to SQL Database

Persist the final date table to a permanent table. Adjust the schema/table name as needed.

In [7]:
-- Write to permanent table (change schema/table name as needed)
IF OBJECT_ID('dbo.DateTable', 'U') IS NOT NULL 
    DROP TABLE dbo.DateTable;

SELECT * 
INTO dbo.DateTable
FROM dbo.DateTableFinal;

## Testing & Validation

Validate fiscal year logic and print sample output.

In [8]:
-- Test: Fiscal year boundary (e.g., July)
SELECT TOP 5 * 
FROM dbo.DateTable 
WHERE MONTH([Date]) = 7 
ORDER BY [Date];

-- Show first few rows
SELECT TOP 3 * 
FROM dbo.DateTable 
ORDER BY [Date];

-- Show last few rows
SELECT TOP 3 * 
FROM dbo.DateTable 
ORDER BY [Date] DESC;